In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cadd_io as cdio
import cadd_main as cdmain
import mdutilities_io as mduio
import mymath as Mmath
import my_plot as myplot
import cadddatadump as cddump
import cadd_mesh2 as cdmesh
import cadd
import os
import sys
import cadd_plot as cdplot
%matplotlib qt
%load_ext autoreload
%autoreload 2

# Preliminaries

## Directories

In [2]:
maindir = 'C:/Users/Varun/Dropbox/Myself/CADD_Code/Tests/CMesh_1/'
uidir = maindir + 'User Inputs/'
fidir = maindir + 'Fortran Inputs/'
dumpdir = maindir + 'Dump Files/'

# C Mesh

## Create

In [3]:
# parameters:
# xmin, xmax, ymin, ymax for middle
# nx, ny for middle
# nx, ny for legs
def c_mesh(boundsmiddle,nmiddle,nlegs):
    mesh = cdmesh.MeshRectangle(boundsmiddle,nmiddle)
    xmin, xmax, ymin, ymax = boundsmiddle
    nx, ny = nmiddle
    nxlegs, nylegs = nlegs
    xstep = (xmax-xmin)/(nx-1)
    ystep = (ymax-ymin)/(ny-1)
    startpointtop = np.array([xmin,ymax])
    endpointtop = np.array([xmin,ymax-ystep*(nylegs-1)])
    startpointbot = np.array([xmin,ymin])
    endpointbot = np.array([xmin,ymin+ystep*(nylegs-1)])
    def add_leg_to_mesh(mesh,startpoint,endpoint):
        ystart = startpoint[1]
        yend = endpoint[1]
        line = cdmesh.Line(startpoint,endpoint)
        line.nodenums = mesh.nodes.search_for_nodes_along_line(line)
        lines = [line]
        for i in range(1,nxlegs):
            xcoordnew = xmin - i*xstep
            startpointnew = np.array([xcoordnew,ystart])
            endpointnew = np.array([xcoordnew,yend])
            line = cdmesh.Line(startpointnew,endpointnew)
            line.nodenums = mesh.nodes.gen_fe_nodes_along_line(line,nylegs)
            lines.append(line)
        for lineprev, linenext in cdmesh.pairwise(lines): # iterate over prev, current
            elements = mesh.elements_std(lineprev.nodenums,linenext.nodenums)
            mesh.add_elements_from_list(elements)
    add_leg_to_mesh(mesh,startpointtop,endpointtop)
    add_leg_to_mesh(mesh,startpointbot,endpointbot)
    return mesh

In [4]:
boundsmiddle = np.array([0,3,-4,4])
nmiddle = [4,9]
nlegs = [6,4]
cmesh = c_mesh(boundsmiddle,nmiddle,nlegs)

## Visualize

In [5]:
cmesh.write_dump_all('cmesh.dump')
simtype = 'fe'
cmeshsim = cdmain.Simulation(simtype,'cmesh_1',uidir,fidir,dumpdir,readinput=False)

In [6]:
cmeshsim.plot_dump_from_file('cmesh.dump',tight=False);

# IO

## Write Nodes, Elements

In [7]:
cmesh.write_user_inputs_all('feelementsmat1','nodes',uidir)

## Read User Inputs

In [9]:
simtype = 'dd'
cmeshsim = cdmain.Simulation(simtype,'cmesh_1',uidir,fidir,dumpdir,nfematerials=1)

## Write Fortran Inputs

In [10]:
cmeshsim.write_fortran_all()

# Dump Outputs

In [15]:
cmeshsim.plot_dump_from_increment(1,fignum=1,tight=False);

In [16]:
cmeshsim.plot_dump_from_increment(1,fignum=2,tight=False);

In [24]:
cmeshsim.plot_dump_from_increment(11,fignum=2,tight=False);

In [13]:
# Compare displacements
b = cmeshsim.data.materials.structlist[0].burgers.val
dispexpected = b/2*np.array([np.cos(np.pi/3),np.sin(np.pi/3)])
print(dispexpected)

[ 0.280615    0.48603944]
